# Data Preprocessing

## Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Importing Dataset

In [2]:
df = pd.read_csv('/content/American Express User Exit Prediction.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

## Handling Missing Data

In [3]:
#If there were any missing values
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(missing_values, np.nan, strategy = 'mean')
#imputer.fix(X[:,0:10])
#X[:,0:10] = imputer.transform(X[:,0:10])

## Encoding Categorical Data

### Gender Column: Label Encoding

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

### Geography Column: One hot Encoding

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## Splitting data into Test set & Training Set


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#ANN

##Initialization

In [8]:
ann = tf.keras.models.Sequential()

###Adding Input layer and first hidden layer

In [9]:
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))

###Adding the Second Hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units = 5, activation = 'relu'))

###Adding Output layer

In [11]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

##ANN Training

In [12]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

###Training on Training Dataset

In [18]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 200)

Epoch 1/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8554 - loss: 0.3536
Epoch 2/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8474 - loss: 0.3560
Epoch 3/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8521 - loss: 0.3517
Epoch 4/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8555 - loss: 0.3449
Epoch 5/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8470 - loss: 0.3595
Epoch 6/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8493 - loss: 0.3496
Epoch 7/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8478 - loss: 0.3574
Epoch 8/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8528 - loss: 0.3550
Epoch 9/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8485 - loss: 0.3496
Epoch 10/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8460 - loss: 0.3631
Epoch 11/200
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8433 - loss: 0.3646
Epoch 12/200
249/249 ━━━━━━━━━━━━━━━━━━━━

##Predictions

Single Predictions

In [19]:
print(ann.predict(sc.transform([[0.0, 1.0, 0.0, 501, 0, 32, 2, 0.0, 4, 1, 54550]])) > 0.5)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[False]]


Prediction on Testing Data

In [20]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[[0 1]
 [0 0]
 [0 0]
 ...
 [0 1]
 [0 0]
 [0 1]]


###Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1519   54]
 [ 238  175]]


0.8529707955689829

Storing the model

In [22]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(ann, file)
